# 2 教程
---
1. MNIST初级教程
2. MNIST高级教程
3. TensorFlow使用指南
4. 卷积神经网络
5. 单词的向量表示
6. 循环神经网络
7. 序列到序列模型
8. Mandelbrot集合
9. 偏微分方程
10. MNIST数据下载
11. 视觉物体识别
12. Deep Dream

# 3 运作方式
1. 综述
2. 变量
3. TensorBoard：可视化学习
4. TensorBoard：图标可视化
5. 数据读取
6. 线程和队列
7. 增加一个Op
8. 自定义数据读取
9. 使用GPUs
10. 共享变量

# 4 资源
1. 其他资源
2. BibTex引用
3. 应用实例
4. 常见问题
5. 术语表
6. 张量的阶，形状，数据类型

## 2.1 Mnist机器学习入门

> ### 1 加载Mnist数据集

```python
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('D:/softfiles/workspace/tensorflow/data/', one_hot=True)
```

> ### 2 softmax回归

**softmax模型**可以用来**给不同的对象分配概率**。在进行模型训练的最后一步都常用softmax来分配概率。

softmax的定义为：
$$softmax(x) = normalize(exp(x))$$
展开等式右边的式子：
$$softmax(x)_i = \frac{exp(x_i)}{\sum_jexp(x_j)}$$



> ### 3 实现回归模型

```python
x = tf.placeholder('float', [None, 784])
```
- x不是一个特定的值，而是一个**占位符**placeholder,只需要在运行的时候赋值即可。

```python
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros[10])
```
一个**Variable**代表一个**可修改的tensor**,存在于tensorflow的用于描述交互性操作的图中。**一般模型参数**都可以用**Variable**表示。

```python
y = tf.nn.softmax(tf.matmul(x, W) + b)
```

> ### 4 训练模型

1. 定义损失函数

定义损失函数评估模型的好坏。常见的成本函数有**交叉熵**。
$$H_{y'}=-\sum_iy'_ilog(y_i)$$
- y是预测的概率分布
- $y'$是实际的分布

**计算交叉熵**
```python
y_ = tf.placeholder('float', [None, 10])
```
所以：
```python
cross_entropy = - tf.reduce_sum(y_ * tf.log(y))
```

2. 反向传播，更新权重值

```python
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
```
这里使用梯度下降算法。

Tensorflow他会在后台给描述你的计算的那张图里面增加一系列的计算操作单元用于实现反向传播算法和梯度下降算法。然后，它返回给你的只是一个单一的操作，当运行这个操作时，它用梯度下降算法训练你的模型，微调你的变量，不断减少成本。

3. 初始化

```python
init = tf.initiallize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    
# 开始训练
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
```
随机抓取训练数据中的100个批次的处理数据点，进行训练。

4. 模型评估

```python
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

accuracy = tf.raduce_mean(tf.cast(correct_prediction, 'float'))

# 测试训练数据，打印正确率
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
```

## 2.2 Mnist优化
---
> ### 1 权重初始化

```python
def weight_variable(shape):
    initial = tf.truncated_mormal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variables(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
```

> ### 2 定义卷积和池化

```python
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
                        
```

> ### 3 第一层卷积

```python
# [5,5,1,32]前两个维度是patch的大小，接着是输入通道的数目，最后是输出的通道数目。
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# [-1, 28, 28, 1]第二三维对应图片的宽和高，最后一维对应图片的颜色通道数。
x_image = tf.reshape(x, [-1, 28, 28, 1])

# 开始卷积和池化操作
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

```
*输出为14 x 14 x 32*

> ### 4 第二层卷积

```python
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
```
*输出为7 x 7 x 64*

> ### 5 密集连接层

```python
# 加入一个有1024个神经元的全连接层
W_fcl = weight_variable([7 * 7 * 64, 1024])
b_fcl = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fcl = tf.nn.relu(tf.matmul(h_pool2_flat, W_fcl) + b_fcl)
```

> ### 6 Dropout

为了减少过拟合，在输出层之前加入dropout。
```python
keep_prob = tf.placeholder('float')
h_fcl_drop = tf.nn.dropout(h_fcl, keep_prob)
```

> ### 7 输出层

```python
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fcl_drop, W_fc2) + b_fc2)
```

> ### 8 训练和评估模型

```python
# 交叉熵
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))

# 梯度下降
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# 判断预测准确性
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))

# 准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

# 初始化
sess.run(tf.initialize_all_variables())

# 开始训练
for i in range(20000):
    batch = mnist.train.next_batch(50)
    # 每100次输出一次准确率
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob: 1.0})
        print('After %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob: 0.5})

# 输出test准确率
print('test accuracy %g' % accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0
}))
```

## 2.3 TensorFlow使用指南
---


> ### 1 数据准备

```python
images_placeholder = tf.placeholder(tf.float32, shape=(batch_size, IMAGE_PIXELS))

label_placeholder = tf.placeholder(tf.int32, shape=(batch_size))
```

> ### 2 构建图表（重点）

```python
def inference():
    # 构建图表, 返回预测结果
    
    # 每一层的操作都创建一个域
#     with tf.name_scope('hidden1') as scope;
#         weights = tf.Variable(tf.truncated_normal([IMAGE_PIXES],stddev=1.0/math.sqrt(float(IMAGE_PIXES))), name='weight')
#         biases = tf.Variable(tf.zeros([hidden1_units]), name='biases')
     
    # 图表的主要操作
    # hidden1 = tf.nn.relu(tf.matmul(images, weights1) + biases1)
    # hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
    # logits = tf.matmul(hidden2, weights) + biases
    pass

def loss():
    # 往inference图表中添加loss
    
    # 处理label数据
#     batch_size = tf.size(labels)
#     labels = tf.expand_dims(labels, 1)
#     indices = tf.expand_dims(tf.range(0, batch_size, 1), 1)
#     concated = tf.concat(1, [indices, labels])
#     onehot_labels = tf.sparse_to_dense(concated, tf,pack([batch_size, NUM_CLASSES]), 1.0. 0.0)
    
#     cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, onehot_labels, name='xentropy')
    
#     loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')

def training():
    # 添加梯度操作,将损失最小化。
    # 从loss函数中获取损失Tensor，将其交给tf.scalar_summary,在与SummaryWriter配合使用，可以向事件文件event file
    # 中生成汇总值summary values。每次写入汇总值时，他都会释放损失Tensor的当前值snapshot value.
    
#     tf.scalar_summary(loss.op.name, loss)
    
    optimizer = tf.train.GradientDescentOptimizer(FLAGS, learning_rate)
    
    # 之后生成一个变量用于保存全局训练步骤global training step的数值，并使用`minimize()`
    # 函数更新系统中的三角权重，增加全局步骤的操作。
    # 这个操作称为`train_op`，是TensorFlow会话session诱发一个完整训练步骤所必须运行的操作。
    
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = optimizer.minimize(loss, global_step=global_step)
    pass
```

> ### 3 训练模型

1. **图表**

```python
# 将所有已经构建的操作都与默认的Graph全局实例关联起来。
with tf.Graph().as_default():
    # 初始化所有variable
    init = tf.initialize_all_variables()
    sess.run(init)
```

2. **会话**


```python
# 构建session用于运行图表
sess = tf.Session()
# or
with tf.Session() as sess:
```

3. **训练循环**

```python
# 开始训练
for step in range(max_step):
    sess.run(train_op)
```

4. **向图表提供反馈**

`fill_feed_dict`函数会查询给定的`DataSet`,索要下一批次的`batch_size`的图像和标签。
```python
images_feed, labels_feed = data_set.next_batch(FLAGS.batch_size)

feed_dict = {
    images_placeholder: images_feed,
    labels_placeholder: labels_feed
}
```

5. **检查状态**

```python
for step in xrange(FLAGS.max_steps):
    feed_dict = fill_feed_dict(data_sets.train, images_placeholder, labels_placeholder)
    
    _, loss_value = sess.run([train_op, loss], feed_dict=feed_dict)
    
    # 每隔100步打印训练状态
    if step % 100 == 0:
        print(xxxxxx)
```

6. **状态可视化**

```python
summary_op = tf.merge_all_summaries()

summary_writer = tf.train.SummaryWriter(FLAGS.trin_dir, graph_def=sess.graph_def)

summary_str = sess.run(summary_op, feed_dict=feed_dict)

summary_writer.add_summary(summary_str, step)
```

7. **保存检查点**

```python
saver = tf.train.Saver()
# 把sess保存到指定文件夹
saver.save(sess, FLAGS.train_dir, global_step=step)

# 重载
saver.restore(sess, FLAGS.train_dir)
```

> ### 4 评估模型

每隔一定的训练步骤，会分别对**训练数据集，验证数据集，测试数据集**进行模型评估。
1. **构建评估图表**

2. **评估图表的输出**

## 2.4 卷积神经网络
---


# 3 运作方式
---

## 3.1 综述
---

> ### 1 Variables

1. **创建variable**

```python
tf.Variable(initial_value=None, trainable=True, collections=None, validate_shape=True, caching_device=None, name=None, variable_def=None, dtype=None, expected_shape=None, import_scope=None, constraint=None)

```

2. **初始化**


- `tf.initialize_all_variables()`并行初始化.同时初始化所有变量，用于所有变量没有嵌套的情形。

```python
weights = tf.Variable(tf.random_normal([784, 200], stddev=0.35), name='wights')
biases = tf.Variable(tf.zeros([200]), name='biases')

init_op = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init_op)
```

- `initialized_value()`用于一个variable嵌套一个variable

```python
weights = tf.Variable(tf.random_normal([...]))

w2 = tf.Variable(weights.initialized_value() * 0.2, name='w2')
```

- `tf.global_variables_initializer()`全局变量初始化


3. **保存、重载**


- 保存

```python
v1 = tf.Variable(..., name='v1')
v2 = tf.Variable(..., name='v2')

init_op = tf.initialize_all_variable()

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    save_path = saver.save(sess, './model.ckpt')
```

- 恢复

```python
v1 = tf.Variable(..., name='v1')
v2 = tf.Variable(..., name='v2')

saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, './model.ckpt')
```

- 显示保存

```python
v1 = tf.Variable(..., name='v1')
v2 = tf.Variable(..., name='v2')

saver = tf.train.Saver({'my_v2': v2})


```

> ### 2 TensorFlow机制 101(缺失)

> ### 3. TensorBoard:学习过程的可视化(ing)

```python
tf
```

> ### 4. TensorBoard:图的可视化(img)

> ### 5. 数据读入

tensorflow程度读取数据一共有三种方法
1. feeding: 在TensorFlow程序运行的每一步，让python代码来供给数据。`tf.placeholder()`
2. 从文件读取数据：在TensorFlow图的起始，让一个输入管线从文件中读取数据。

```python
# 获取文件列表,输出为字符串张量列表
tf.train.match_filenames_once(pattern, name=None)

# 创建文件队列
tf.train.string_input_producer(string_tensor, num_epochs=None, shuffle=True, seed=None, capacity=32, shared_name=None, name=None, cancel_op=None)
```

3. 预加载数据：在TensorFlow图中定义常量或变量来保存所有数据。

> ### 6. 线程和队列

1. 队列

```python
tf.FIFOQueue()
tf.RandomShuffleQueue()
```

2. Coordinator

```python

```
3. QueueRunner

> ### 7. 添加新的Op

> ### 8 自定义数据的Readers

> ### 9 使用GPUs

> ### 10 共享变量

```python
tf.get_variable(name, shape=None, dtype=None, initializer=None)

tf.variable_scope(scope_name)

# 常用的初始化方法
tf.constant_initializer()
tf.random_uniform_initializer()
tf.random_normal_initializer()


```